<a href="https://colab.research.google.com/github/pmb06d/rating_analysis_scripts/blob/master/Compilation_for_GOTS7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up the Google Drive Client

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import os
import re


# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# Personal Drive --> 17XQvo2BbXl2dedMl39YX3ni3IZjKeiLg
# HBO Drive --> 1oO-ElX2tGLBdKIuuWeSrtFKbZ7Rwcbcq

file_list = drive.ListFile(
    {'q': "'14boNlN820kxvym_VjQwVbU9wACboPq0p' in parents"}).GetList()

# filenames
titles = [file['title'] for file in file_list]

# Countries from the filenames
countries = [re.findall('^\w+',i)[0] for i in titles]

# drive IDs
id_list = [file['id'] for file in file_list]

In [0]:
# Some helper functions

# Target standarizer
def target_normalizer(target):
    import re
        
    age_regex = re.compile('[PWM][0-9\-\+]+.+')
    age_regex2 = re.compile('[Ss]*[ubs\+]*\w+\s*\-*Universe+')
    
    try:
      clean_str = age_regex.findall(target)[0].replace('04','')
    except:
      try:
        clean_str = (age_regex2.findall(target)[0].replace('-',' '))
      except:
        clean_str = target

    # hardcoded to move this analysis along
    clean_str = clean_str.replace(' (ABC+C)','')
    clean_str = clean_str.replace('Subs+Comm','Pay')
    clean_str = clean_str.replace('+',' ')

    return(clean_str)

# Skipper function for IBOPE MW output
def skipper(file):
    with open(file, encoding='latin1') as f:
        lines = f.readlines()
        # get list of all possible lines starting with quotation marks
        num = [i for i, l in enumerate(lines) if l.startswith('"')]
        
        # if not found value return 0 else get first value of list subtracted by 1
        num = 0 if len(num) == 0 else num[0]
        return(num)


In [4]:
import pandas as pd
from tqdm import tqdm

def clean_features(test_list):
  temp = []
  for i in test_list:
    try:
      temp.append(re.findall('^\w+[\s\%\#]\w*',i)[0])
    except:
      temp.append(i)
  return(temp)

df_list = []

for id, country in tqdm(zip(id_list, countries), total=len(countries)):
  # Storing the file into a dataframe
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('Filename.csv')  
  df = pd.read_csv('Filename.csv', sep = ';',encoding='latin1' )
  df['Region']= country
  df.columns = clean_features(list(df))
  df_list.append(df)

100%|██████████| 297/297 [04:01<00:00,  2.02it/s]


In [5]:
df_stack = pd.concat(df_list, ignore_index=True, sort=False)

# Some extra preprocessing for the stack
df_stack = df_stack.loc[df_stack['Date'].notna(),:]
df_stack = df_stack.iloc[:,1:-1]

df_stack.loc[:,'Target_NM'] = df_stack['Target'].apply(target_normalizer)

df_stack['Target_NM'].unique()

array(['P18-49', 'P25-49 High Med', 'Pay Universe'], dtype=object)

In [0]:
# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

file_list2 = drive.ListFile(
    {'q': "'1fwJT7QKP9eWGhzaMA60pVhXGvTuEfNoW' in parents"}).GetList()

# filenames
titles = [file['title'] for file in file_list2]

# Countries from the filenames
countries = [re.findall('^\w+',i)[0] for i in titles]

# drive IDs
id_list = [file['id'] for file in file_list2]

In [7]:
df_list2 = []

for id, country in tqdm(zip(id_list, countries), total=len(countries)):
  # Storing the file into a dataframe
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('Filename.csv')  
  df = pd.read_csv('Filename.csv', sep = ';',skiprows = skipper('Filename.csv'), encoding='latin1' )
  df['Region']= country
  df.columns = clean_features(list(df))
  df_list2.append(df)

100%|██████████| 5/5 [00:02<00:00,  1.82it/s]


In [8]:
df_stack2 = pd.concat(df_list2, ignore_index=True, sort=False)

df_stack2 = df_stack2.loc[:,['Region',
                             'Target',
                             'Channel',
                             'Description',
                             'Desc2',
                             'Desc3',
                             'Desc4',
                             'Date',
                             'Week Day',
                             'TimeBand',
                             'Start Time',
                             'End Time',
                             'Rat%']]

df_stack2 =  df_stack2.loc[df_stack2['Target'].notna(),:]

df_stack2.loc[:,'Target_NM'] = df_stack2['Target'].apply(target_normalizer)

df_stack2['Target_NM'].unique()

array(['Pay Universe'], dtype=object)

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df_stack.head()

print('\n')
df_stack2.head()

,Target,Channel,TimeBand,Date,Start Time,Shr%,Rat#,Rat%,Region,Target_NM
0,Subs+Comm_P18-49,Cinemax,9 (NINE),"Sun Sep 01, 2019",06:09:00,3.15,9.6433,0.1164,Colombia,P18-49
1,Subs+Comm_P18-49,Cinemax,BRIDESMAIDS,"Sun Sep 01, 2019",12:27:00,0.28,3.2887,0.0397,Colombia,P18-49
2,Subs+Comm_P18-49,Cinemax,BROKEBACK MOUNTAIN,"Sun Sep 01, 2019",25:46:00,0.00,0.0000,0.0000,Colombia,P18-49
3,Subs+Comm_P18-49,Cinemax,CARNAGE,"Sun Sep 01, 2019",06:00:00,4.17,9.0145,0.1088,Colombia,P18-49
4,Subs+Comm_P18-49,Cinemax,HERBIE: FULLY LOADED,"Sun Sep 01, 2019",14:53:00,1.66,19.5180,0.2355,Colombia,P18-49


,Region,Target,Channel,Description,Desc2,Desc3,Desc4,Date,Week Day,TimeBand,Start Time,End Time,Rat%,Target_NM
0,Brazil,Live / Pay-Universe 4+,Cinemax,17 AGAIN,FILM,Comédia,WARNER,"Sat Sep 14, 2019",Saturday,Afternoon,14:26:00,16:12:00,0.1014,Pay Universe
1,Brazil,Live / Pay-Universe 4+,Cinemax,17 AGAIN,FILM,Comédia,WARNER,"Fri Sep 20, 2019",Friday,Morning,10:45:00,12:34:00,0.0801,Pay Universe
2,Brazil,Live / Pay-Universe 4+,Cinemax,17 AGAIN,FILM,Comédia,WARNER,"Fri Sep 20, 2019",Friday,Primetime,20:06:00,22:00:00,0.0698,Pay Universe
3,Brazil,Live / Pay-Universe 4+,Cinemax,2010: THE YEAR WE MAKE CONTACT,FILM,Ficçao Científica,WARNER,"Mon Sep 02, 2019",Monday,Late,29:47:00,30:00:00,0.1148,Pay Universe
4,Brazil,Live / Pay-Universe 4+,Cinemax,2010: THE YEAR WE MAKE CONTACT,FILM,Ficçao Científica,WARNER,"Tue Sep 03, 2019",Tuesday,Morning,06:00:00,08:00:00,0.0827,Pay Universe


In [20]:
df_stack = df_stack.rename(columns={'TimeBand': 'Description'})

matching_stack = df_stack2.loc[:,['Region',
                                  'Description',
                                  'Desc2',
                                  'Desc3',
                                  'Desc4',
                                  'Date',
                                  'Week Day',
                                  'TimeBand',
                                  'Start Time',
                                  'End Time']]
                               
full_data = df_stack.merge(matching_stack, how='left', left_on=['Region','Start Time','Date','Description'], right_on=['Region','Start Time','Date','Description'])

# Check the shapes to make sure the join didn't add rows
print(df_stack.shape, 'vs', full_data.shape)

(859209, 10) vs (859209, 16)


In [0]:
keepers = ['Target_NM',
           'Region',
           'Channel',
           'Date',
           'Week Day',
           'TimeBand',
           'Description',
           'Desc2',
           'Desc3',
           'Desc4',
           'Start Time',
           'End Time',
           'Shr%',
           'Rat#',
           'Rat%']

full_data = full_data.loc[:,keepers]

In [25]:
full_data.head()

,Target_NM,Region,Channel,Date,Week Day,TimeBand,Description,Desc2,Desc3,Desc4,Start Time,End Time,Shr%,Rat#,Rat%
0,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Morning,9 (NINE),FILM,Animacion Aventura,WHILAND INT'L S.A.,06:09:00,07:39:00,3.15,9.6433,0.1164
1,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Morning,BRIDESMAIDS,FILM,Comedia,UNIVERSAL CITY STUDIO PRODUCTION LLP,12:27:00,14:53:00,0.28,3.2887,0.0397
2,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Primetime,BROKEBACK MOUNTAIN,FILM,Drama,UNIVERSAL CITY STUDIO PRODUCTION LLP,25:46:00,28:26:00,0.00,0.0000,0.0000
3,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Morning,CARNAGE,FILM,Comedia Drama,STAR TV S.A.,06:00:00,06:09:00,4.17,9.0145,0.1088
4,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Afternoon,HERBIE: FULLY LOADED,FILM,Aventura Comedia Acción,BUENA VISTA,14:53:00,16:51:00,1.66,19.5180,0.2355


In [0]:
#!pip install --upgrade -q gspread

# This package can read google sheets
import gspread

gc = gspread.authorize(GoogleCredentials.get_application_default())
categories = gc.open_by_url('https://docs.google.com/spreadsheets/d/1tpaCzJpDf_hx8mlyKzP18FNdigfHDpzdg8WKBaWmtrg/edit?usp=sharing')
categories = categories.sheet1

# Put into a dataframe
categories = pd.DataFrame(categories.get_all_values(), columns=['MW_Name','Category1','Category2','Distributor'])
categories = categories.iloc[1:len(categories),]
del categories.index.name

In [39]:
# join it to the main dataset
full_data2 = full_data.copy()

full_data2.loc[:,'Channel'] = full_data2['Channel'].str.replace(' (MF)', '', regex=False)
full_data2.loc[:,'Channel'] = full_data2['Channel'].str.replace('_MF', '', regex=False)

full_data2 = full_data2.merge(categories, how='left', left_on='Channel', right_on='MW_Name')

# Check for missing channels
#full_data2.loc[full_data2['Category1'].isna(),'Channel'].unique()

# Exclude the channels we don't want
exclude = ['Virtual','Children']
full_data2 = full_data2.loc[~(full_data2['Category1'].isin(exclude)),:]

full_data2['Rank_Rat%'] = full_data2.groupby(['Target_NM', 'Region', 'Date', 'Description', 'Start Time'])['Rat%'].rank(ascending=False,method='first')

full_data2.head()

,Target_NM,Region,Channel,Date,Week Day,TimeBand,Description,Desc2,Desc3,Desc4,Start Time,End Time,Shr%,Rat#,Rat%,MW_Name,Category1,Category2,Distributor,Rank_Rat%
0,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Morning,9 (NINE),FILM,Animacion Aventura,WHILAND INT'L S.A.,06:09:00,07:39:00,3.15,9.6433,0.1164,Cinemax,M&S,Movie,HBO LAG,4.0
1,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Morning,BRIDESMAIDS,FILM,Comedia,UNIVERSAL CITY STUDIO PRODUCTION LLP,12:27:00,14:53:00,0.28,3.2887,0.0397,Cinemax,M&S,Movie,HBO LAG,29.0
2,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Primetime,BROKEBACK MOUNTAIN,FILM,Drama,UNIVERSAL CITY STUDIO PRODUCTION LLP,25:46:00,28:26:00,0.00,0.0000,0.0000,Cinemax,M&S,Movie,HBO LAG,27.0
3,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Morning,CARNAGE,FILM,Comedia Drama,STAR TV S.A.,06:00:00,06:09:00,4.17,9.0145,0.1088,Cinemax,M&S,Movie,HBO LAG,6.0
4,P18-49,Colombia,Cinemax,"Sun Sep 01, 2019",Sunday,Afternoon,HERBIE: FULLY LOADED,FILM,Aventura Comedia Acción,BUENA VISTA,14:53:00,16:51:00,1.66,19.5180,0.2355,Cinemax,M&S,Movie,HBO LAG,10.0


In [41]:
# Test the ranker

full_data2.loc[(full_data2['Target_NM']=='P18-49')
               & (full_data2['Region']=='Colombia')
               & (full_data2['Description']=='BRIDESMAIDS')
               & (full_data2['Start Time']=='12:27:00'),
               ['Rank_Rat%','Channel','Rat%']].sort_values(by=['Rat%'],ascending = False).reset_index(drop=True).head(10)

,Rank_Rat%,Channel,Rat%
0,1.0,FOX Channel,0.7373
1,2.0,TNT,0.4237
2,3.0,Paramount,0.3918
3,4.0,InfoClaro,0.3758
4,5.0,Space,0.3704
5,6.0,ESPN2,0.2277
6,7.0,FX,0.2199
7,8.0,Win Sports,0.2185
8,9.0,Fox Sports,0.2011
9,10.0,Sony,0.1740


In [0]:
from google.colab import files

# e.g. save pandas output as csv
full_data2.to_csv('GOT_vs_movies.csv', index=False)

files.download('GOT_vs_movies.csv')